---
# Seminar 3 - Grundlagen Ablaufplanung
---





---
### Einlesen der Klassen aus erster Veranstaltung 
---

In [1]:
!pip install plotly

In [2]:
import plotly

In [3]:
# files from first session
from InputData import InputData
from OutputData import OutputJob
# Additionally
import numpy


---
### 1. Codierung und Bewertung einer Lösung
---
Im Rahmen Ihrer Werkstudententätigkeit bei Dr. Best sollen Sie die Produktionsleiterin Traudel Teufel bei der Ablaufplanung unterstützen. Aktuell wird die Fertigungsreihenfolge für die nächste Woche immer freitags festgelegt. Aufgrund der zunehmenden Komplexität fällt es Frau Teufel immer schwerer, gute Ablaufpläne zu generieren. Derzeit nutzt sie Ihre lange Erfahrung im Bereich der Zahnpastafertigung und erzeugt manuell eine geeignete Bearbeitungsreihenfolge, welche Sie an die Schichtleiter weitergibt.

In der nächsten Woche sollen 11 Aufträge abgearbeitet werden. Die Inputdaten sind Ihnen in der Datei "InputFlowshopSIST.json" gegeben. Traudel Teufel plant mit folgender Reihenfolge:
**6-5-7-4-8-3-9-2-10-1-11**

Diese Auftragsreihenfolge soll auf allen Maschinen eingehalten werden (Permutation Flow Shop). Leider kann die Produktionsleiterin nicht einschätzen, wie gut Ihre Lösung tatsächlich ist. Da sie von Ihren Programmierfähigkeiten gehört hat, bittet sie Sie, diese Lösung zu bewerten. Anschließend sollen Sie außerdem Auskunft darüber geben, wann welcher Auftrag auf welcher Maschine laut Plan bearbeitet wird.

#### a.) 
Sie überlegen, dass es sinnvoll ist, eine eigene Klasse für Lösungen anzulegen, wenn Sie im folgenden verschiedene Lösungen erzeugen wollen. Deshalb schreiben Sie als erstes eine Klasse **Solution**, welche ein Dictionary mit allen Aufträge als **OutputJobs** sowie die Fertigungsreihenfolge als Liste **Permutation** enthält. Zudem sollen im Konstruktor die Attribute Makespan, TotalTardiness und TotalWeightedTardiness angelegt und zunächst auf -1 gesetzt werden. Nachdem Sie die Klasse definiert haben, bietet es sich an, als erstes Objekt dieser Klasse die von Frau Teufel vorgeschlagene Lösung mit Hilfe der Inputdaten in der Datei "InputFlowshopSIST.json" zu erzeugen. Nennen Sie diese erste Lösung **DevilSolution**.


In [4]:


class Solution():
    def __init__(self, joblist, permutation):
        self.Makespan = -1
        self.TotalTardiness = -1
        self.TotalWeightedTardiness = -1
        self.Permutation = permutation

        self.OutputJobs = {}
        for jobId, job in enumerate(joblist):
            self.OutputJobs[jobId] = OutputJob(job)

    def __str__(self):
        return f'The Permutation {self.Permutation} results in a Makespan of {self.Makespan}'

    def SetPermutation(self, permutation):
        self.Permutation = permutation


#### Führen Sie im Anschluss folgenden Code aus ####
data = InputData("InputFlowshopSIST.json")
Permutation = [x-1 for x in [6,5,7,4,8,3,9,2,10,1,11]]
DevilSolution = Solution(data.InputJobs, Permutation) 
print(DevilSolution)

The Permutation [5, 4, 6, 3, 7, 2, 8, 1, 9, 0, 10] results in a Makespan of -1


In [5]:

from InputData import *
from OutputData import *

class Solution:
    def __init__(self, joblist, permutation):
        # Solution 的属性
        self.Makespan = -1
        self.TotalTardiness = -1
        self.TotalWeightedTardiness = -1
        self.Permutation = permutation
        self.OutputJobs = {}

        for jobId, job in enumerate(joblist):
            self.OutputJobs[jobId] = OutputJob(job)
            # 注意看OutputJob类的属性，能用于计算Makespan
            # OutputJob的用意在于，把输入数据中的订单信息进一步细化并计算
            # 注意看OutputJob构造器的参数表，需要填入的是 InputData("....").InputJobs这个列表的每一个元素
    
    def __str__(self):# 输出函数，用来展示当前permutation造成的总工作时长Makespan
        return f'The Permutation {self.Permutation} results in a Makespan of {self.Makespan}'

    def SetPermutation(self, permutation):
        self.Permutation = permutation

# 构造solution类需要哪些参数？joblist和permutation
# 现在已经给出了订单生产顺序，只需要找出joblist即可
# joblist 来源于InputData对象的InputJobs属性


# 导入数据
data = InputData("InputFlowshopSIST.json")

# 调整生产序列，每个元素 -1 才能得到python中的序列
Permutation = [x - 1 for x in [6,5,7,4,8,3,9,2,10,1,11]]

# 用现有的参数构造Solution对象
DevilSolution = Solution(data.InputJobs, Permutation)
print(DevilSolution)

    


The Permutation [5, 4, 6, 3, 7, 2, 8, 1, 9, 0, 10] results in a Makespan of -1


#### b.) 
Um die Qualität einer Lösung einschätzen zu können, sollten Sie als nächstes eine Bewertungsfunktion für ein gegebenes Solution Objekt schreiben. Diese Methode sollte für eine gegebene Reihenfolge (Permutation) allen Aufträgen Start- und Endzeitpunkte zuweisen unter Beachtung, dass jede Maschine nur einen Auftrag zur selben Zeit bearbeiten kann und ein Auftrag nicht gleichzeitig auf mehreren Maschinen bearbeitet werden kann. Die Rüstzeiten können Sie dabei zunächst vernachlässigen. Am Ende der Einplanung können Sie für das Solution Objekt noch das Attribut Makespan festlegen.

Da die erstellte Funktion zur Bewertung einer Lösung benötigt wird, erachten Sie es als sinnvoll, diese der Klasse EvaluationLogic anzuhängen.

Abschließend können Sie Ihre neu entwickelten Methoden testen, indem Sie die von Frau Teufel vorgeschlagene Lösung **DevilSolution** bewerten.

In [6]:
! pip install -U ipykernel

In [7]:
class EvaluationLogic:    
    def DefineStartEnd(self, currentSolution):    #把现有解决方案输入进函数中  
        #####
        # schedule first job: starts when finished at previous stage
        firstJob = currentSolution.OutputJobs[currentSolution.Permutation[0]]
        firstJob.EndTimes = numpy.cumsum([firstJob.ProcessingTime(x) for x in range(len(firstJob.EndTimes))])
        firstJob.StartTimes[1:] = firstJob.EndTimes[:-1]
        #####
        # schedule further jobs: starts when finished at previous stage and the predecessor is no longer on the considered machine
        for j in range(1,len(currentSolution.Permutation)):
            currentJob = currentSolution.OutputJobs[currentSolution.Permutation[j]]
            previousJob = currentSolution.OutputJobs[currentSolution.Permutation[j-1]]
            # first machine
            currentJob.StartTimes[0] = previousJob.EndTimes[0]
            currentJob.EndTimes[0] = currentJob.StartTimes[0] + currentJob.ProcessingTime(0)
            # other machines
            for i in range(1,len(currentJob.StartTimes)):
                currentJob.StartTimes[i] = max(previousJob.EndTimes[i], currentJob.EndTimes[i-1])
                currentJob.EndTimes[i] = currentJob.StartTimes[i] + currentJob.ProcessingTime(i)
        #####
        # Save Makespan and return Solution
        currentSolution.Makespan = currentSolution.OutputJobs[currentSolution.Permutation[-1]].EndTimes[-1]

EvaluationLogic().DefineStartEnd(DevilSolution)
print(DevilSolution.Makespan)

8922


In [8]:
import numpy

class EvaluationLogic:
    def DefineStartEnd(self, currentSolution):

        ####
        # 安排第一个订单：在前一阶段完成后开始。
        firstJob = currentSolution.OutputJobs[currentSolution.Permutation[0]] #currentSolution.Permutation[0] 调用solution对象的permutation属性，然后选择排在第一位的job
        firstJob.EndTimes = numpy.cumsum([firstJob.ProcessingTime(x) for x in range(len(firstJob.EndTimes))])
        firstJob.StartTimes[1: ] = firstJob.EndTimes[ : -1]
            # 第一个生产的订单由于前面没有订单，所有其EndTimes就是累积的加工时间。在第一台机器上的开始时间是初始值0，第二台机器的开始时间是第一台机器结束时间---无缝衔接

        ####
        # 安排其他的订单j: 在每台机器上的开始时间取决于两方面：1.上一个订单j-1是否已经被这个机器i加工完 2. 这个订单j是否在上一台机器i-1上加工完
        for j in range(1, len(currentSolution.Permutation)):
            currentJob = currentSolution.OutputJobs[currentSolution.Permutation[j]]
            previousJob = currentSolution.OutputJobs[currentSolution.Permutation[j-1]]

            # 第一台机器，第j个订单
            currentJob.StartTimes[0] = previousJob.EndTimes[0]
            currentJob.EndTimes[0] = currentJob.StartTimes[0] + currentJob.ProcessingTime(0)

            # 其他机器上处理订单j的开始和结束时间
            for i in range(1, len(currentJob.StartTimes)):
                currentJob.StartTimes[i] = max(previousJob.EndTimes[i], currentJob.EndTimes[i-1])
                currentJob.EndTimes[i] = currentJob.StartTimes[i] + currentJob.ProcessingTime(i)

        ####
        # 计算当前解决方案的总时长 --> 排在最后的那个订单在最后一台机器上的结束时间
        currentSolution.Makespan = currentSolution.OutputJobs[currentSolution.Permutation[-1]].EndTimes[-1]


EvaluationLogic().DefineStartEnd(DevilSolution)
print(DevilSolution.Makespan)

8922


Erwarteter Output:

    8922

#### c.)
Damit alle Mitarbeiter in der Fertigung detailiert über den Ablaufplan informiert werden können, soll in einer .csv Datei für alle Aufträge aufgelistet werden, wann diese bearbeitet werden sollen. Die Tabelle soll dabei die folgenden Spalten enthalten:

|Machine  |Job      |Start_Setup |End_Setup  |Start    |End 	 |
|---------|---------|------------|-----------|---------|---------|
| 1 	  | 1 	    | 0	         | 0	     | 132 	   | 481 	 |
| 1  	  | 2 	    | 0   	     | 0	     | 0 	   | 132 	 |
| ...	  | ... 	| ...        | ... 	     | ... 	   | ...	 | 

Schreiben Sie eine Methode WriteSolToCsv(), welche eine gegebene Lösung (Instanz der Klasse Solution) in eine csv Datei schreibt. Nutzen Sie dabei das Modul **csv**. Erzeugen Sie anschließend eine Ausgabe von **DevilSolution**. Da die Ausgabe nur mit einem Solution Objekt erfolgen kann, sollten die Methode an die Klasse Solution angehangen werden.



In [9]:
import csv

def WriteSolToCsv(self, fileName):  #sollten die Methode an die Klasse Solution angehangen werden.
    
    with open( fileName, "w" ) as csvFile:
        csv_writer = csv.writer(csvFile)

        csv_head = ['Machine', 'Job', 'Start_Setup', 'End_Setup', 'Start', 'End']  #写入列名
        csv_writer.writerow(csv_head)  # writerow 需要一个列表

        for job in self.OutputJobs.values():
            for i in range(len(job.Operations)):
                csv_writer.writerow([i+1, job.JobId, job.StartSetups[i], job.EndSetups[i], job.StartTimes[i], job.EndTimes[i] ])


setattr(Solution, "WriteSolToCsv", WriteSolToCsv)
DevilSolution.WriteSolToCsv("DevilSolution.csv")

        
    


#### d.)
Nachdem Sie aus der Fertigung hören, dass einige Mitarbeiter Probleme beim Verständnis der csv Ausgabe haben, machen Sie sich auf die Suche nach geeigneten Grafikmodulen. Erfreulicherweise stoßen Sie schnell auf die Methode **timeline()** aus dem Modul **plotly.express**. Zudem hatte scheinbar schon eine anderer Programmierer das gleiche Problem, weshalb Ihnen jetzt ein Python Skript vorliegt, welches Sie direkt zur grafischen Darstellung nutzen können. 

Laden Sie das Skript **Gantt.py** und nutzen Sie die Methode **ganttChart**, um DevilSolution grafisch darzustellen. 

---
### 2. Dispatching Rules zur Erstellung von Lösungen
---
Trotz aller Erfahrung ist sich Traudel Teufel unsicher bezüglich der Qualität ihrer Lösung (DevilSolution). Immerhin gibt es 39916800 mögliche Permutationen für die 11 Aufträge. Aus diesem Grund überlegen Sie, wie sich konstruktiv weitere Lösungen erzeugen lassen, um die Güte der bisherigen Lösung einzuschätzen und eventuell eine bessere Lösung zu finden. 

#### a.)
Um überhaupt ein Gefühl für die Verteilung möglicher Ablaufpläne zu erhalten, lassen sich mit der wohl einfachsten Entscheidungsregel **Random Order of Service** zunächst verschiedene Lösungen erzeugen. Das Vorgehen besteht darin, dass der nächste Auftrag immer zufällig gewählt wird. Schreiben Sie deshalb eine Methode **ROS()**, welche zufällig "x" Fertigungsreihenfolgen erzeugt und die beste zurückgibt. Der Parameter "x" soll dabei der Funktion ebenso wie ein Startwert und die Stammdaten der Aufträge übergeben werden. Bei der Generierung zufälliger Permutationen kann Ihnen das Modul Numpy sicher wieder behilflich sein.

In [10]:
import numpy as np
from InputData import *

def ROS(joblist, x, seed):  # stammdaten ueber geben: joblist

    np.random.seed(seed)
    tempSolution = Solution(joblist, 0) # 0 is initial
    bestCmax = np.inf # unendless

    for i in range(x):
       tempPermutation = np.random.permutation(len(joblist))  # anzahl von jobs ubergeben
       tempSolution.SetPermutation(tempPermutation)

       EvaluationLogic().DefineStartEnd(tempSolution)

       if (tempSolution.Makespan < bestCmax):
           bestCmax = tempSolution.Makespan
           bestPerm = tempPermutation

    bestSol = Solution(joblist, bestPerm)
    EvaluationLogic().DefineStartEnd(bestSol)

    return bestSol

data = InputData("InputFlowshopSIST.json")
ROSsolution = ROS(data.InputJobs, 10, 2021)
print(ROSsolution)




The Permutation [ 3  0  4  2  5  1  7 10  9  6  8] results in a Makespan of 7812


In [11]:
import numpy as np

def ROS(jobList, x, seed):
    np.random.seed(seed)

    tmpSolution = Solution(jobList, 0)
    bestCmax = np.inf # 无穷大的浮点数

    for i in range(x): 
        tmpPermutation = np.random.permutation(len(jobList)) # 随机生成与订单数量对应的排列
        tmpSolution.SetPermutation(tmpPermutation)

        EvaluationLogic().DefineStartEnd(tmpSolution) # 这一步在得出Solution对象后一定要做，否则Makespan不会被计算

        if (tmpSolution.Makespan < bestCmax):  # 典型的比较大小算法
            bestCmax = tmpSolution.Makespan
            bestPermutation = tmpPermutation

    bestSolution = Solution(jobList, bestPermutation)  # 实例化最佳方案
    EvaluationLogic().DefineStartEnd(bestSolution)  # 计算总时长

    return bestSolution  # 返回最优解

ROSSolution = ROS(data.InputJobs, 10, 2022)
print(ROSSolution)


ROS_1k = ROS(data.InputJobs, 1000, 2022)
print(ROS_1k)


The Permutation [ 7  8  4  3  9  6  5  0  2  1 10] results in a Makespan of 8210
The Permutation [ 7  2 10  1  8  9  4  0  6  5  3] results in a Makespan of 7175


In [12]:
ROS_10k = ROS(data.InputJobs, 10000, 2022)
print(ROS_10k)

The Permutation [ 7  4  8  2  6  3  0 10  1  5  9] results in a Makespan of 7038


In [13]:
ROS_100k = ROS(data.InputJobs, 100000, 2022)
print(ROS_100k)

The Permutation [ 7  4  8  2  6  3  0 10  1  5  9] results in a Makespan of 7038


Erwarteter Output:

Reihenfolge \[3, 0, 4, 2, 5, 1, 7, 10, 9, 6, 8\] resultiert in Makespan 7812

#### b.)
Ein Mitarbeiter aus der Fertigung weist Sie darauf hin, dass Sie bei 11 Aufträgen auch alle Permutationen berechnen könnten und anschließend die Beste bestimmen können. Sie wollen deshalb eine Funktion **checkAllPermutations()** schreiben, die Ihnen nach Überprüfung aller Reihenfolgen diejenige zurückgibt, die zum besten Makespan führt. Wie lange dauert diese Rechnung?

In [14]:
def checkAllPermutations():
    

SyntaxError: unexpected EOF while parsing (4095751800.py, line 2)

Erwarteter Output:

    Rechenzeit beträgt 111.684 Minuten <br>
    Beste Lösung mit Makespan 7038 <br>
    Beste Reihenfolge ist: 7 2 0 4 6 10 3 5 8 1 9

#### c.)
Nachdem eine vollständige Enumeration selbst bei kleinen Problemen langfristig keine Alternative darstellt, schlagen Sie vor, das Problem mit statischen Einplanungsregeln zu lösen. Frau Teufel ist von dieser Idee begeistert und möchte gern, dass Sie die folgenden Regeln implementieren: <br>
* FCFS (First Come First Serve)
* SPT (Shortest Processing Time)
* LPT (Longest Processing Time) 

Schreiben Sie für jede dieser Regeln eine Methode. Die Funktionen sollen jeweils ein Objekt der Klasse **Solution** ausgeben.



In [15]:
def FirstComeFirstServe(jobList):
    tmpPermutation = [x for x in range(len(jobList))]
    tmpSolution = Solution(jobList, tmpPermutation)
    EvaluationLogic().DefineStartEnd(tmpSolution)

    return tmpSolution

FCFSSol = FirstComeFirstServe(data.InputJobs)
print(FCFSSol)


The Permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 9298


In [16]:
def FirstComeFirstServe(jobList):
    tmpPermutation = [x for x in range(len(jobList))]
    tmpSolution = Solution(jobList, tmpPermutation)

    EvaluationLogic().DefineStartEnd(tmpSolution)

    return tmpSolution

FCFSSolution = FirstComeFirstServe(data.InputJobs)
print(FCFSSolution)



The Permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 9298


Erwarteter Output:

    The permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 9298

In [17]:
def ShortestProcessingTime(jobList):
    jobPool = [] #存放sume der Processingtime, tuple
    for i in range(len(jobList)):
        jobPool.append((i, sum(jobList[i].ProcessingTime(x) for x in range(len(jobList[i].Operations)))))
    jobPool.sort(key= lambda t: t[1])
    tmpPermutation = [x[0] for x in jobPool]

    tmpSolution = Solution(jobList, tmpPermutation)
    EvaluationLogic().DefineStartEnd(tmpSolution)

    return tmpSolution


SPTSol = ShortestProcessingTime(data.InputJobs)
print(SPTSol)

The Permutation [0, 7, 3, 5, 8, 2, 1, 9, 4, 6, 10] results in a Makespan of 8848


In [18]:
def ShortestProcessingTime(jobList, allMachines = False):
    jobPool = [] #存放sume der Processingtime, tuple
    if allMachines:
        for i in range(len(jobList)):
            jobPool.append((i, sum(jobList[i].ProcessingTime(x) for x in range(len(jobList[i].Operations)))))

    else:
        for i in range(len(jobList)):
            jobPool.append((i, jobList[i].ProcessingTime(0)))


    jobPool.sort(key= lambda t: t[1])
    tmpPermutation = [x[0] for x in jobPool]

    tmpSolution = Solution(jobList, tmpPermutation)
    EvaluationLogic().DefineStartEnd(tmpSolution)

    return tmpSolution


SPTSol = ShortestProcessingTime(data.InputJobs)
print(SPTSol)

The Permutation [2, 7, 8, 0, 3, 4, 6, 10, 1, 5, 9] results in a Makespan of 7718


In [19]:
def ShortestProcessingTime(jobList, allMachines = False):

    jobPool = []
    if allMachines:
        for i in range(len(jobList)):
            jobPool.append((i, sum(jobList[i].ProcessingTime(x) for x in range(len(jobList[i].Operations)))))

    else:
        for i in range(len(jobList)):
            jobPool.append((i, jobList[i].ProcessingTime(0)))

    jobPool.sort(key = lambda t: t[1])

    tmpPermutation = [x[0] for x in jobPool]
    tmpSolution = Solution(jobList, tmpPermutation)

    EvaluationLogic().DefineStartEnd(tmpSolution)

    return tmpSolution

# 只比较第一台机器的时间
SPTSloution = ShortestProcessingTime(data.InputJobs)
print(SPTSloution)

# 比较所有机器的加工时间
SPRSolutionAll = ShortestProcessingTime(data.InputJobs, True)
print(SPRSolutionAll)




The Permutation [2, 7, 8, 0, 3, 4, 6, 10, 1, 5, 9] results in a Makespan of 7718
The Permutation [0, 7, 3, 5, 8, 2, 1, 9, 4, 6, 10] results in a Makespan of 8848


In [20]:
def LongestProcessingTime(jobList, allMachines = False):
    jobPool = [] #存放sume der Processingtime, tuple
    if allMachines:
        for i in range(len(jobList)):
            jobPool.append((i, sum(jobList[i].ProcessingTime(x) for x in range(len(jobList[i].Operations)))))

    else:
        for i in range(len(jobList)):
            jobPool.append((i, jobList[i].ProcessingTime(0)))


    jobPool.sort(key= lambda t: -t[1])
    tmpPermutation = [x[0] for x in jobPool]

    tmpSolution = Solution(jobList, tmpPermutation)
    EvaluationLogic().DefineStartEnd(tmpSolution)

    return tmpSolution


SPTSol = LongestProcessingTime(data.InputJobs)
print(SPTSol)

The Permutation [9, 5, 1, 6, 10, 4, 3, 0, 8, 7, 2] results in a Makespan of 10649


Erwarteter Output:

    The permutation [2, 7, 8, 0, 3, 4, 6, 10, 1, 5, 9] results in a Makespan of 7718

Erwarteter Output:

    The permutation [9, 5, 1, 6, 10, 4, 3, 0, 8, 7, 2] results in a Makespan of 10649

#### d.) NEH Heuristik
Auch wenn Sie durch die Dispatching Rules sehr einfach und schnell Lösungen generieren können, sind Sie doch von der Lösungsgüte etwas enttäuscht. Selbst die SPT Regel führt zu fast 10% Abweichung von der optimalen Lösung. Aus diesem Grund erachten Sie es als sinnvoll, aufwendigere konstruktive Verfahren zu analysieren. Schnell stellen Sie fest, dass die NEH Heuristik zu den am meisten verwendeten Ansätzen gehört und häufig sehr gute Startlösungen liefert. Deshalb wollen Sie im Folgenden selbst eine Methode **NEH()** schreiben und das Verfahren implementieren.

In [21]:
from copy import deepcopy

def DetermineBestInsertion(solution,   jobToInsert):
    ###
    # insert job at front of permutation
    solution.Permutation.insert(0, jobToInsert)
    bestPermutation = deepcopy(solution.Permutation)
    
    EvaluationLogic().DefineStartEnd(solution)
    bestCmax = solution.Makespan

    ###
    # swap job i to each position and check for improvement
    lengthPermutation = len(solution.Permutation) - 1
    for j in range(0, lengthPermutation):
        solution.Permutation[j], solution.Permutation[j + 1] = solution.Permutation[j+1], solution.Permutation[j]
        EvaluationLogic().DefineStartEnd(solution)
        if(solution.Makespan < bestCmax):
            bestCmax = solution.Makespan
            bestPermutation = [x for x in solution.Permutation]

    solution.Makespan = bestCmax
    solution.Permutation = bestPermutation

def NEH(jobList):
    jobPool = []
    tmpPerm = []
    bestCmax = 0
    # Calculate sum of processing times and sort
    for i in range(len(jobList)):
        jobPool.append((i,sum(jobList[i].ProcessingTime(x) for x in range(len(jobList[i].Operations)))))
    jobPool.sort(key=lambda x: x[1], reverse=True)

    # Initalize input
    tmpNEHOrder = [x[0] for x in jobPool]
    tmpPerm.append(tmpNEHOrder[0])
    tmpSolution = Solution(jobList,tmpPerm)

    # Add next jobs in a loop and check all permutations
    for i in range(1,len(tmpNEHOrder)):
        # add next job to end and calculate makespan
        DetermineBestInsertion(tmpSolution, tmpNEHOrder[i])
    
    return tmpSolution

NEHSol = NEH(data.InputJobs)    
print(NEHSol)



The Permutation [7, 0, 4, 8, 2, 10, 3, 6, 5, 1, 9] results in a Makespan of 7038


In [22]:
from copy import deepcopy


def DetermineBestInsertion(solution, jobToInsert):

    ###
    # 把订单jobToInsert插入到当前Permutation的首位。（当前Permutation只有一个元素，即初始NEH排列的第一个元素）
    solution.Permutation.insert(0, jobToInsert) # insert函数，在索引位置插入数据，索引后面的往后顺移一位。初始肯定在第0位插入
    bestPermutation = deepcopy(solution.Permutation) #创建一个新的排列，不能修改原本的排列

    EvaluationLogic().DefineStartEnd(solution)
    bestCmax = solution.Makespan

    ###
    # 把位于首位的jobToInsert逐位右移，并比较Makespan
    lengthPermutation = len(solution.Permutation) - 1
    for j in range(0, lengthPermutation): # 只看这个函数的定义，现在只安排了两个订单。一个是输入的，另一个是插入的
        solution.Permutation[j], solution.Permutation[j+1] = solution.Permutation[j+1], solution.Permutation[j]

        EvaluationLogic().DefineStartEnd(solution)
        if (solution.Makespan < bestCmax):
            bestCmax = solution.Makespan
            bestPermutation = [x for x in solution.Permutation] # 这里思考为什么不能把solution.Permutation 直接赋给bestPerm

    solution.Makespan = bestCmax
    solution.Permutation = bestPermutation

def NEH(jobList):
    jobPool = []
    tmpPerm = []
    bestCmax = 0

    # 计算总加工时间并降序排列 --> 即生成NEH初始序列
    for i in range(len(jobList)):
        jobPool.append((i, sum(jobList[i].ProcessingTime(x) for x in range(len(jobList[i].Operations)))))

        jobPool.sort(key = lambda x: x[1], reverse = True)

    # 初始化输入， 从NEH的第一步开始执行
    tmpNEHOrder = [x[0] for x in jobPool] # x是个元组，x[1]是元组的第二个元素，也就是订单总的加工时间。实际上总时长在这里没有意义,因为顺序会被打乱
    tmpPerm.append(tmpNEHOrder[0]) # 在初始情况下，临时排列只有一个元素，跟DetermineBestInsertion的输入对上了
    tmpSolution = Solution(jobList, tmpPerm)  # 这里存疑，Solution类能否接受一个不完整的tmpPerm？

    # 借助循环，往当前的解决方案中插入后面的元素
    for i in range(1, len(tmpNEHOrder)):
        # 利用DetermineBestInsertion函数插入元素并计算时长
        DetermineBestInsertion(tmpSolution, tmpNEHOrder[i])

    return tmpSolution


NEHSloution = NEH(data.InputJobs)
print(NEHSloution)

The Permutation [7, 0, 4, 8, 2, 10, 3, 6, 5, 1, 9] results in a Makespan of 7038


Erwarteter Output:

    The permutation [7, 0, 4, 8, 2, 10, 3, 6, 5, 1, 9] results in a Makespan of 7038

#### e.)

Glücklich darüber, dass Sie mit Hilfe der NEH eine sehr gute Lösung gefunden haben. Wollen Sie Ihre implementierten konstruktiven Lösungsansätze abschließend in einer eigenen Klasse zusammenfassen. Nennen Sie die neue Klasse **ConstructiveHeuristic**. Damit Sie zukünftig einfach auf die einzelnen Methoden zugreifen können, entscheiden Sie sich, eine Hauptroutine zu schreiben, über welche Sie unter Angabe der jeweiligen konstruktiven Heuristik auf die Methoden zugreifen können.

In [23]:
class ConstructiveHeuristic:
    def Run(self, inputData, solutionMethod):
        if solutionMethod == 'FCFS':
            solution = FirstComeFirstServe(inputData.InputJobs)
        elif solutionMethod == 'SPT':
            solution = ShortestProcessingTime(inputData.InputJobs)
        elif solutionMethod == 'LPT':
            solution = LongestProcessingTime(inputData.InputJobs)
        elif solutionMethod == 'ROS':
            solution = ROS(inputData.InputJobs, 100, 2022)
        elif solutionMethod == 'NEH':
            solution = NEH(inputData.InputJobs)
        else:
            print('Unknown constructive solution method!')

        return solution


print(ConstructiveHeuristic().Run(data, 'FCFS'))
print(ConstructiveHeuristic().Run(data, 'SPT'))
print(ConstructiveHeuristic().Run(data, 'LPT'))
print(ConstructiveHeuristic().Run(data, 'ROS'))
print(ConstructiveHeuristic().Run(data, 'NEH'))



The Permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] results in a Makespan of 9298
The Permutation [2, 7, 8, 0, 3, 4, 6, 10, 1, 5, 9] results in a Makespan of 7718
The Permutation [9, 5, 1, 6, 10, 4, 3, 0, 8, 7, 2] results in a Makespan of 10649
The Permutation [ 7  0  8  2  4  9  6  3 10  5  1] results in a Makespan of 7190
The Permutation [7, 0, 4, 8, 2, 10, 3, 6, 5, 1, 9] results in a Makespan of 7038


In [ ]:

    def Run(inputData, solutionMethod):
        if solutionMethod == 'FCFS':
            solution = FirstComeFirstServe(inputData.InputJobs)
        elif solutionMethod == 'SPT':
            solution = ShortestProcessingTime(inputData.InputJobs)
        elif solutionMethod == 'LPT':
            solution = LongestProcessingTime(inputData.InputJobs)
        elif solutionMethod == 'ROS':
            solution = ROS(inputData.InputJobs)
        elif solutionMethod == 'NEH':
            solution = NEH(inputData.InputJobs)
        else:
            print('Unknown constructive solution method!')

        return solution


print(Run(data, 'FCFS'))

NameError: name 'Run' is not defined

In [ ]:
def Run(inputData, solutionMethod):
    if solutionMethod == 'FCFS':
        solution = FirstComeFirstServe(inputData.InputJobs)
        elif solutionMethod == 'SPT':
            solution = ShortestProcessingTime(inputData.InputJobs)
        elif solutionMethod == 'LPT':
            solution = LongestProcessingTime(inputData.InputJobs)
        elif solutionMethod == 'ROS':
            solution = ROS(inputData.InputJobs)
        elif solutionMethod == 'NEH':
            solution = NEH(inputData.InputJobs)
        else:
            print('Unknown constructive solution method!')

        return solution

---
### 3. Liefertreue 
--- 
Ihre bisherigen Lösungsversuchen konzentrieren sich auf eine möglichst effiziente Fertigung, indem die Kapazitäten möglichst gut ausgelastet werden. Allerdings stellt für Dr. Best auch die Liefertreue ein wichtiges Kriterium dar, damit die Kunden immer pünklich ihre Zähne putzen können. Deshalb soll im Folgenden **Total Tardiness** als alternatives Zielkriterium betrachtet werden.

a) 
Welche gesamten Verspätungen ergeben sich mit den bisherigen konstruktiven Lösungsansätzen **NEH**, **SPT** und **FCFS**? Schreiben Sie zuvor zwei Methoden in der Klasse EvaluationLogic, welche Total Tardiness und Total Weighted Tardiness berechnen.

In [ ]:
def CalculateTardiness(self, currentSolution):
        totalTardiness = 0
        for key in currentSolution.OutputJobs:
            if(currentSolution.OutputJobs[key].EndTimes[-1] - currentSolution.OutputJobs[key].DueDate > 0): # verspaetung
                currentSolution.OutputJobs[key].Tardiness = currentSolution.OutputJobs[key].EndTimes[-1] - currentSolution.OutputJobs[key].DueDate
                totalTardiness += currentSolution.OutputJobs[key].EndTimes[-1] - currentSolution.OutputJobs[key].DueDate
        currentSolution.TotalTardiness = totalTardiness       

setattr(EvaluationLogic,"CalculateTardiness",CalculateTardiness)

def CalculateWeightedTardiness(self, currentSolution):
    totalWeightedTardiness = 0
    for key in currentSolution.OutputJobs:
        if(currentSolution.OutputJobs[key].EndTimes[-1] - currentSolution.OutputJobs[key].DueDate > 0):
            currentSolution.OutputJobs[key].Tardiness = currentSolution.OutputJobs[key].EndTimes[-1] - currentSolution.OutputJobs[key].DueDate
            totalWeightedTardiness += (currentSolution.OutputJobs[key].EndTimes[-1] - currentSolution.OutputJobs[key].DueDate) * currentSolution.OutputJobs[key].TardCost
    currentSolution.TotalWeightedTardiness = totalWeightedTardiness

setattr(EvaluationLogic,"CalculateWeightedTardiness",CalculateWeightedTardiness)


EvaluationLogic().CalculateTardiness(NEHSloution)
print("NEH Regel führt zu gesamten Verspätungen von: "+ str(NEHSol.TotalTardiness))
EvaluationLogic().CalculateTardiness(FCFSSolution)
print("FCFS Regel führt zu gesamten Verspätungen von: "+ str(FCFSSol.TotalTardiness))
EvaluationLogic().CalculateTardiness(SPTSloution)
print("SPT Regel führt zu gesamten Verspätungen von: "+ str(SPTSloution.TotalTardiness))

NEH Regel führt zu gesamten Verspätungen von: -1


NameError: name 'FCFSSol' is not defined

Erwarteter Output:

    NEH Regel führt zu gesamten Verspätungen von: 18152
    FCFS Regel führt zu gesamten Verspätungen von: 26812
    SPT Regel führt zu gesamten Verspätungen von: 21866

In [24]:
def CalculateTardiness(self, currentSolution):

    totalTardiness = 0
    for key in currentSolution.OutputJobs: # 直接对字典进行遍历，遍历的是字典的key
        if(currentSolution.OutputJobs[key].EndTimes[-1] > currentSolution.OutputJobs[key].DueDate):
            currentSolution.OutputJobs[key].Tardiness = currentSolution.OutputJobs[key].EndTimes[-1] - currentSolution.OutputJobs[key].DueDate
            totalTardiness += currentSolution.OutputJobs[key].EndTimes[-1] - currentSolution.OutputJobs[key].DueDate
    currentSolution.TotalTardiness = totalTardiness

setattr(EvaluationLogic, 'CalculateTardiness', CalculateTardiness)



def CalculateWeightedTardiness(self, currentSolution):
    
    totalWeightedTardiness = 0
    for key in currentSolution.OutputJobs:
        if(currentSolution.OutputJobs[key].EndTimes[-1] > currentSolution.OutputJobs[key].DueDate):
            currentSolution.OutputJobs[key].Tardiness = currentSolution.OutputJobs[key].EndTimes[-1] - currentSolution.OutputJobs[key].DueDate
            totalWeightedTardiness += (currentSolution.OutputJobs[key].EndTimes[-1] - currentSolution.OutputJobs[key].DueDate) * currentSolution.OutputJobs[key].TardCost
    currentSolution.TotalWeightedTardiness = totalWeightedTardiness

setattr(EvaluationLogic, 'CalculateWeightedTardiness', CalculateWeightedTardiness)




# 下面进行计算，记住，上面两个函数都被写在EvaluationLogic中，要想调用这两个函数必须先将EvaluationLogic实例化
EvaluationLogic().CalculateTardiness(NEHSloution)
print(f'NEH rule leads to total delays of: {NEHSloution.TotalTardiness}')
EvaluationLogic().CalculateTardiness(FCFSSolution)
print(f'FCFS rule leads to total delays of: {FCFSSolution.TotalTardiness}')
EvaluationLogic().CalculateTardiness(SPTSloution)
print(f'SPT rule leads to total delays of: {SPTSloution.TotalTardiness}')


EvaluationLogic().CalculateWeightedTardiness(NEHSloution)
print(f'NEH rule leads to total weighted delays of: {NEHSloution.TotalWeightedTardiness}')
EvaluationLogic().CalculateWeightedTardiness(FCFSSolution)
print(f'FCFS rule leads to total weighted delays of: {FCFSSolution.TotalWeightedTardiness}')
EvaluationLogic().CalculateWeightedTardiness(SPTSloution)
print(f'SPT rule leads to total weighted delays of: {SPTSloution.TotalWeightedTardiness}')



NEH rule leads to total delays of: 18152
FCFS rule leads to total delays of: 26812
SPT rule leads to total delays of: 21866
NEH rule leads to total weighted delays of: 3630400
FCFS rule leads to total weighted delays of: 5362400
SPT rule leads to total weighted delays of: 4373200


#### b)
Da die bisherigen konstruktiven Lösungsansätze noch nicht auf Tardiness ausgerichtet sind, vermuten Sie, dass noch Potential zur Verbesserung besteht. Deshalb wollen Sie im Folgenden die Entscheidungsregel **Earliest Due Date** zur Lösungserzeugung nutzen. Schreiben Sie dafür eine entsprechende Funktion. Wie hoch sind hierbei die Verspätungen?

In [30]:
def EarliestDueDate(jobList):
    tmpPermutation = sorted(range(len(jobList)), key = lambda x: jobList[x].DueDate)
    tmpSolution = Solution(jobList, tmpPermutation)

    # EvaluationLogic().DefineStartEnd(tmpSolution)
    EvaluationLogic().CalculateTardiness(tmpSolution)

    return (tmpSolution)


EDDSol = EarliestDueDate(data.InputJobs)
print(f'EDD rule leads to total weighted delays of: {EDDSol.TotalTardiness}')




EDD rule leads to total weighted delays of: 0


In [35]:
from InputData import *
def EarliestDueDate(jobList):
    tmpPermutation = sorted(range(len(jobList)), key = lambda x: jobList[x].DueDate)
    tmpSolution = Solution(jobList, tmpPermutation)

    EvaluationLogic().DefineStartEnd(tmpSolution)
    EvaluationLogic().CalculateTardiness(tmpSolution)

    return tmpSolution


EDDSol = EarliestDueDate(data.InputJobs)
print(f'EDD rule leads to total weighted delays of: {EDDSol.TotalTardiness}')

EDD rule leads to total weighted delays of: 15859


Erwarteter Output:
   
    EDD Regel führt zu gesamten Verspätungen von: 15859